In [112]:
import numpy as np
import os
import pandas as pd
import time
from sklearn.linear_model import LinearRegression
pd.options.display.float_format = '{:.5f}'.format
np.set_printoptions(suppress=True)

In [90]:
swing_dir = ['swing/back_cut','swing/back_drive','swing/back_short','swing/back_smash','swing/fo_cut','swing/fo_drive','swing/fo_short','swing/fo_smash']
sensor_axis = ['AX','AY','AZ']

file_name_dir = []

for root,dirs,files in os.walk('swing'):
    for fname in files:
        full_name = os.path.join(root,fname)
        file_name_dir.append(full_name)

print(len(file_name_dir))

12331


In [91]:
from tqdm import tqdm
total_li = []
label_li = []

def label_encoding(st):
    if 'back_cut' in st:
        return 1
    elif 'back_smash' in st:
        return 2
    elif 'back_drive' in st:
        return 3
    elif 'back_short' in st:
        return 4
    elif 'fo_cut' in st:
        return 5
    elif 'fo_smash' in st:
        return 6
    elif 'fo_drive' in st:
        return 7
    elif 'fo_short' in st:
        return 8

for file in tqdm(file_name_dir):
    label_li.append(label_encoding(file))
    data = open(file,'r').read().split('\n')
    data.pop(0) # remove trash index
    index = data.pop(0) # get real_index
    real_data = []
    
    for tmp in range(len(data)):
        if data[tmp] == '':
            continue
        real_data.append(data[tmp].split(','))
        
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    
    tmp_li = []
    for i in range(len(df)):
        tmp = []
        tmp.append((df['AX'][i]))
        tmp.append((df['AY'][i]))
        tmp.append((df['AZ'][i]))
        tmp_li.append(tmp)
    total_li.append(tmp_li)


    
    
    

100%|██████████| 12331/12331 [01:02<00:00, 195.97it/s]


In [92]:
print(len(total_li),len(label_li))

12331 12331


In [93]:
total_li = np.array(total_li)
nsamples,nx,ny = total_li.shape
total_li = total_li.reshape((nsamples,nx*ny))

In [94]:
from sklearn.model_selection import cross_val_score,train_test_split

In [95]:
x_train,x_test,y_train,y_test = train_test_split(total_li,label_li,test_size=0.2,random_state=0)

In [107]:
import math
import sklearn.metrics as sklm
from sklearn.linear_model import LogisticRegression
lreg = LogisticRegression()
lin = lreg.fit(x_train,y_train)
lin_pred = lin.predict(x_test)
print('rmse : ',math.sqrt(sklm.mean_squared_error(y_test,lin_pred)))

/home/younghwan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/younghwan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


rmse :  0.8393446205070105


/home/younghwan/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [117]:
w = lin.coef_
b = lin.intercept_
hit_count = 0
miss_count = 0

for nu in range(200):
    res=np.matmul(w,x_test[nu])+b
    if np.abs(y_test[nu]-np.argmax(res))<=1.8:
        
        hit_count+=1
    else:
        print(res,'y_res : ',y_test[nu])
        miss_count+=1
print(hit_count)
print(miss_count)

[-64.86913494  -6.18815667  -1.64616186  -1.84862819 -24.2434184
  -2.92001286 -12.69163094  -3.04555851] y_res :  4
[  7.78962205  -7.09574534 -16.54812516 -17.95439409 -40.78742616
  -3.03401331 -10.74559448  -9.48055702] y_res :  6
[-86.1847639    1.18266975   0.78897815 -18.20164208 -14.88570658
  -5.34764916  -5.74386774  -6.97345939] y_res :  3
[ -58.41703273   -9.37673546  -26.31480224  -17.32740165 -104.92405808
    0.93709106    0.27137077   -1.68942522] y_res :  7
[-115.17240956   -1.85679463   -8.20534725   -8.75568363  -10.27565751
   -1.54007313  -12.81946375   -8.50423861] y_res :  2
[-71.07987885   1.24108493 -23.57518277  -8.78892929 -38.27911676
  -0.80203482  -1.19190554  -1.41530605] y_res :  8
[-36.29582841   0.54879381  -6.98236997  -5.8737477  -42.06059981
 -11.1145269   -6.69466348 -14.32040784] y_res :  3
[ -9.30036347 -11.0118401  -12.77336949  -9.27327026 -15.2624764
  -1.9934485   -5.30307681  -2.38843623] y_res :  1
[-105.71707543    6.5549952     0.15253553

[ 2.46107364e+12 -2.79909738e+12  2.78534034e+12 -9.92691630e+10
 -6.13790592e+12 -5.44064215e+12  6.34007623e+11  4.16724982e+12
  2.74325724e+12  1.96229938e+12  4.41918666e+12 -2.83365375e+12
  5.76669355e+11  1.79568696e+12 -4.16250240e+12  1.78927679e+12
  5.63968657e+12 -2.33383528e+12  4.57538935e+12 -4.48603013e+12
  4.08151079e+12  8.12013223e+11 -6.37181472e+11  3.95741104e+12
  9.78671233e+12  7.53029739e+12  6.25615188e+11 -1.27347030e+12
 -1.27961496e+13  2.28229502e+12 -3.80965958e+12 -3.67123154e+12
  3.71167450e+12  1.00803488e+12 -7.24404001e+11 -3.57612996e+12
 -9.81612011e+11 -6.68634658e+12  2.82881968e+12  2.50286886e+12
  1.36029680e+12 -1.67890147e+11 -4.86670891e+12 -4.95903358e+12
 -6.93374258e+12  3.29706197e+11  3.52281787e+12  3.46232000e+12
 -9.78791697e+11 -8.21659439e+11 -5.87602143e+11 -6.21152035e+12
 -1.28506305e+12  2.40519701e+12 -6.79785873e+12  6.26086643e+12
  1.31963203e+12 -4.17242495e+11  5.68888932e+11 -3.99517313e+12
 -2.03120639e+12  2.26074